In [1]:
import numpy as np

In [2]:
import tensorflow as tf
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
import random

In [4]:
import nltk

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noure\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import json

In [7]:
import pickle

In [8]:
import warnings

In [9]:
from tensorflow.python.framework import ops

In [10]:

from nltk.tokenize import word_tokenize

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\noure\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
#punkt is a pre_trained model for tokenization

In [13]:
from nltk.stem.lancaster import LancasterStemmer

In [14]:
stemmer=LancasterStemmer()

In [15]:
warnings.filterwarnings("ignore")

In [16]:
words=[] # a collection of terms that includes the tokenized words

In [17]:
classes=[] # a collection of terms that includes the tokenized words with their tags

In [18]:
documents=[] # a collection that includes the tag of the words

In [19]:
ignore_words=['?']

In [20]:
#importing data
print('importing the data')
with open('intents.json') as json_data:
    Intents=json.loads(json_data.read())

importing the data


In [21]:
print(Intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there', 'Hey', 'Hola', 'Hello'], 'responses': ['Hello', 'hi ', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Till next time', 'bbye'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand']}, {'tag': 'options', 'patterns': ['How you could help me', 'What can  you do', 'What help you provide', 'What support is offered', 'who are you', 'what is your name'], 'responses': ['I am a medical chatbot.I am designed to provide you with medical information and assistance! ']}, {'tag': 'Accute Branchitis', 'patterns': ['branchitis', '

In [22]:
#looping through the data to covert them to words classes documents and ignore words
for intent in Intents['intents']:
    for pattern in intent['patterns']:
        #adding tokenized words in the patterns to words list
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        #add to the documents corpus (it contains tuples of each word and its tag)
        documents.append((w,intent['tag']))
        #adding the tag to the corpus if it does not exist
        if intent['tag'] not in classes:
            classes.append((intent['tag']))

In [23]:
for i in Intents.keys():
    print(i)


intents


In [24]:
print(words)

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', 'Hey', 'Hola', 'Hello', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Till', 'next', 'time', 'bbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', 'What', 'can', 'you', 'do', 'What', 'help', 'you', 'provide', 'What', 'support', 'is', 'offered', 'who', 'are', 'you', 'what', 'is', 'your', 'name', 'branchitis', 'Wheezing', 'feeling', 'short', 'of', 'breath', 'blocked', 'nose', 'fever', 'coughing', 'loss', 'of', 'taste', 'loss', 'of', 'smell', 'loss', 'of', 'taste', 'and', 'smell', 'headache', 'diarrhea', 'fatigue', 'coughing', 'covid', 'Whats', 'up', 'Wazzup', 'How', 'are', 'you', 'how', 'are', 'you', 'sup', 'How', 'you', 'doing', 'diabetes', 'increased', 'urine', 'output', 'excessive', 'thirst', 'slow', 'healing', 'wounds', 'increased', 'urine', 'skin', 'problems', 'nosebleeds', 'hypertension', 'headaches', 'blurred', 'vision', 'y

In [25]:
print(classes)

['greeting', 'goodbye', 'thanks', 'options', 'Accute Branchitis', 'COVID', 'whatsup', 'Diabetes', 'hypertension', 'insult', 'activity', 'High Colesterol', 'Hepatitis', "Crohn's desease", 'autism', 'anemia', 'nicetty', 'no', 'Heart Desease', 'migraine', 'broken bones', 'sun stroke', 'malaria', 'Influenza', 'kidney stones']


In [26]:
#stemming and lowering
words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words ]

In [27]:
#removing duplication
classes=sorted(list(set(classes)))

In [28]:
print(len(documents),"documents")
print(len(classes),"classes")
print(len(words),"words")

134 documents
25 classes
346 words


In [29]:
#creating the data for the model
training=[]
outputs=[]
#creating an empty list for the output
output_empty=[0]*len(classes)

In [30]:
#creating a training set bag of words
for doc in documents:
    #initializing the bag of words
    bag=[]
    #list of tokenized words for the pattern
    pattern_words=doc[0]
    #stem each word
    pattern_words=[stemmer.stem(w.lower()) for w in pattern_words]
    #creating the bag_of_words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    #output is a 0 for each tag and 1 for current tag
    output_row=list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag,output_row])

In [31]:
print(training)

[[[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
#shuffling randomly and converting to numpy array for faster processing
random.shuffle(training)
training=np.array(training)

In [33]:
#creating train and test lists
train_x=list(training[:,0])
train_y=list(training[:,1])

In [34]:
#building neural network for our chatbot to be contextuel
#resetting graph data

In [35]:
ops.reset_default_graph()

In [36]:
net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)
print('training..')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
training..


In [37]:
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

In [38]:
#training the model...

In [39]:
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)

Training Step: 16999  | total loss: 0.14360 | time: 0.048s
| Adam | epoch: 1000 | loss: 0.14360 - acc: 0.9252 -- iter: 128/134
Training Step: 17000  | total loss: 0.14675 | time: 0.056s
| Adam | epoch: 1000 | loss: 0.14675 - acc: 0.9202 -- iter: 134/134
--


In [40]:
#saving the model

In [41]:
model.save('model.tflearn')

INFO:tensorflow:C:\Users\noure\nlp\medical_chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\noure\nlp\medical_chatbot\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\noure\nlp\medical_chatbot\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\noure\nlp\medical_chatbot\model.tflearn.meta
INFO:tensorflow:100


In [42]:
print('pickle is also saved')
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y} , open("training_data","wb"))

pickle is also saved


In [43]:
print('loading pickle')
data=pickle.load(open("training_data","rb"))
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

loading pickle


In [44]:
with open('intents.json') as json_data:
    Intents=json.loads(json_data.read())
    print('loading the model')

loading the model


In [45]:
#loading the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\noure\nlp\medical_chatbot\model.tflearn


In [46]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [47]:
def clean_text(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[stemmer.stem(w.lower()) for w in sentence_words]
    
    return sentence_words

In [48]:
def bag_of_words(sentence,words,show_details=False):
    sentence_words=clean_text(sentence)
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("found in bag :%s" %w)
        return(np.array(bag))
ERROR_THRESHOLD=0.25
print('ERROR_THRESHOLD=0.25')
def classify(sentence):
    results=model.predict([bag_of_words(sentence,words)])[0] #get the probability from the model
    results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD] #exclude the results that are below threshold
    results.sort(key=lambda x :x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1])) #list of tuples of intents and probabilities
    return return_list

        
    
    

ERROR_THRESHOLD=0.25


In [49]:
def response(sentence,userID='123',show_details=False):
    results=classify(sentence)
    if results:
        for i in Intents['intents']:
            if i['tag']==results[0][0]:
                return print(random.choice(i['responses'])) #randpm response from high order probability
        results.pop(0)
            

In [50]:
print('the model is ready')

the model is ready


In [ ]:
while True:
    input_data=input("you:")
    answer=response(input_data)
    answer

you:hi
hi 
you:what s up
All good..What about you?
you:cholesterol symptoms
High cholesterol can be inherited, but it's often the result of unhealthy lifestyle choices, which make it preventable and treatable. A healthy diet, regular exercise and sometimes medication can help reduce high cholesterol.and it has no symptoms a blood test is the only way to know if you have it
you:autism
Based on the description above, you may feel that your child should be evaluated for autism. If that's the case:Read a little more about the symptoms of autism to be sure that you're accurately understanding how autism differs from other developmental challenges.T
you:delayed language skills
Based on the description above, you may feel that your child should be evaluated for autism. If that's the case:Read a little more about the symptoms of autism to be sure that you're accurately understanding how autism differs from other developmental challenges.T
you:coughing and fatigue
COVID-19 is caused by infectio